In [ ]:
import json
import logging
import random
import uuid

import boto3
from botocore.exceptions import ClientError
s3 = boto3.resource('s3')


logger = logging.getLogger(__name__)


class ObjectWrapper(object):
    
    """Encapsulates S3 object actions."""
    def __init__(self, s3_object):
        """
        :param s3_object: A Boto3 Object resource. This is a high-level resource in Boto3
                          that wraps object actions in a class-like structure.
        """
        self.object = s3_object
        self.key = self.object.key
        return None 
    
    def put(self, data):
        """
        Upload data ti the object.
        """
        put_data = data
        if isinstance(data, str):
            try:
                put_data = open(data, 'rb')
            except IOError:
                logger.exception("Expected file name or binary data, got '%s'.", data)
                raise   
        try:
            self.object.put(Body=put_data)
            self.object.wait_until_exists()
        except ClientError:
            logger.exception(
             "Couldn't put object '%s' to bucket '%s'.", self.object.key,
                self.object.bucket_name
            )            
        finally:
            if getattr(put_data, 'close', None):
                put_data.close()
                

def main():
    bucket = s3.Bucket('pandan.lambdaa')
    object_key = 'doc-example-object'
    objectwrapper = ObjectWrapper(bucket.Object(object_key))
    objectwrapper.put(__file__)

if __name__ == "__main__":
    main()
    